Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. Create a convolutional neural network that trains to 100% accuracy on these images, which cancels training upon hitting training accuracy of >.999

In [47]:
import tensorflow as tf
import os
import zipfile
from os import path, getcwd, chdir

In [48]:
path = f"{getcwd()}/datasets/happy-or-sad.zip"

zip_ref = zipfile.ZipFile(path, 'r')
zip_ref.extractall("datasets/h-or-s")
zip_ref.close()

In [49]:
DESIRED_ACCURACY = 0.999

In [50]:
class MyCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy')) >= DESIRED_ACCURACY:
            print("\nReached 99.8% accuracy so cancelling training!")
            self.model.stop_training = True

In [51]:
callbacks = MyCallback()

In [52]:
model = tf.keras.models.Sequential([
        # First convolution
        # 150x150 resolution and last channel with 3 for RGB value from 0 to 255 each
        tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
        tf.keras.layers.MaxPooling2D(2,2),
        # Second convolution
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        # Third convolution
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        # Flatten to feed to DNN
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        # Only 1 output => happy | sad
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

In [61]:
from tensorflow.keras.optimizers import RMSprop

model.compile(
    loss='binary_crossentropy',
    optimizer=RMSprop(learning_rate=0.001),
    metrics=['accuracy']
)

In [62]:
num_samples = 80
batch_size = 32
epochs = 15

In [63]:
# This code block should create an instance of an ImageDataGenerator called train_datagen
# And a train_generator by calling train_datagen.flow_from_directory

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255.0) # Your Code Here

# Please use a target_size of 150 X 150.
train_generator = train_datagen.flow_from_directory(
    'datasets/h-or-s',
    target_size=(150,150),
    batch_size=batch_size,
    class_mode='binary'
)
# Expected output: 'Found 80 images belonging to 2 classes'

Found 80 images belonging to 2 classes.


In [64]:
# This code block should call model.fit_generator and train for
# a number of epochs.
# model fitting
# fit_generator deprecated, instead using fit
history = model.fit(
    train_generator,
    steps_per_epoch=(num_samples // batch_size),
    epochs=epochs,
    verbose=1,
    callbacks=[callbacks]
)

Epoch 1/15
2/2 [==============================] - 1s 197ms/step - loss: 9.3479 - accuracy: 0.3281
Epoch 2/15
2/2 [==============================] - 0s 195ms/step - loss: 0.7908 - accuracy: 0.5781
Epoch 3/15
2/2 [==============================] - 0s 205ms/step - loss: 0.8767 - accuracy: 0.5625
Epoch 4/15
2/2 [==============================] - 0s 202ms/step - loss: 0.6802 - accuracy: 0.5469
Epoch 5/15
2/2 [==============================] - 0s 203ms/step - loss: 0.6256 - accuracy: 0.5833
Epoch 6/15
2/2 [==============================] - 0s 202ms/step - loss: 0.7731 - accuracy: 0.5417
Epoch 7/15
2/2 [==============================] - 0s 143ms/step - loss: 0.5804 - accuracy: 0.5833
Epoch 8/15
2/2 [==============================] - 0s 209ms/step - loss: 0.4482 - accuracy: 0.8281
Epoch 9/15
2/2 [==============================] - 0s 135ms/step - loss: 0.5406 - accuracy: 0.5417
Epoch 10/15
2/2 [==============================] - 0s 200ms/step - loss: 0.3783 - accuracy: 0.7917
Epoch 11/15
2/2 [==

In [66]:
print(history.history['accuracy'][-1])

1.0
